# Ant Colony Optimization

### Ant class: the “agents” that will be traversing the graph.


### Ant colony: a colony of ants. It is responsible for moving ants to their starting node as well as prompting the ants to move to the next node on their “journey”.


### Ant graph: the graph our agents will be traversing over


### Task: the task the ants will complete ("watch" movies and rate them)

In [27]:
import ast 
import random 
import numpy as np
import pandas as pd
import copy

In [28]:
#lets think about our cost function for a bit
# we want to use the users ratings to 

In [29]:
class Ant: 
    trail = []
    visited_jobs = []
    review_list = []
    current_film = 0;
    
    def __init__(self, review_list):
        self.review_list = sorted(review_list, key=lambda x: x[1])
    
    def trail_len():
        return len(trail)
    
    def get_trail():
        return trail
    
    def get_current():
        return current_film
    
    def has_visited (i):
        visited_jobs[i]
        
    def watch_movie (movieID):
        current_film = movieID
        trail.append(movieID)
        
    def clear ():
        trail = []
        visited_jobs = []
        current_film = 0;
        
        
    #get user review for specific movie returns a neutral rating of 2.5 if it does not exist
    def get_task_cost(movieID):
        return next((y for x, y in review_list if x == movieID ), 2.5)
    
    def get_best_film():
        return review_list.pop(0) 

In [30]:
class AntColony:
    
    ants = []
    alpha = 0
    beta = 0 
    
    def __init__(self,antID_list,filename,alpha =0.5,beta = 0.4 ):
        user_reviews =  pd.read_csv(filename)
        
        for ID in antID_list:
            rev = ast.literal_eval(user_reviews.loc[user_reviews['userId'] == ID]["ratings_list"].tolist()[0])
            new_ant = Ant(rev)
#             new_ant.watch_movie(new_ant.get_best_film()[0])
            ants.append(new_ant)
        
        
    def prob_fx (rating, current_weight,sum_weights):
        prob = ((current_weight **alpha) * (rating ** beta)) / sum_weights
        return prob
    
    def move_all_ants (edge_weights):
        for ant in ants:
            movie_id = determine_next_movie(ant,edge_weights[ant.get_current ()])
            ant.watch_movie(movie_id)
            
        ## will be used for global pheremone updating 
        ##return all_ants_tours
        
    def return_all_tours():
        tours = []
        for ant in ants:
            tours.append(ant.get_trail())
        
        ## will be used for global pheremone updating 
        return tours
    
    #here we use the index of the ant rather than its ID 
    def get_ant_rating (antIndex,movieID):
        return ants[antIndex].get_task_cost(movieID)
    
    def clear ():
        for ant in ants:
            ant.clear()
        
            
    def determine_next_movie(ant,edge_weights):
        current_movie = ant.get_current ()
        trail = get_trail()
        temp_weights = {}
        probs = {}
        sum_cost = 0 
        
        #removes all nodes we have already visited
        for ew in edge_weights.keys():
            if (ew not in trail):
                temp_weights [ew] = edge_weights[ew]
                
        # getting 
        for vertex in temp_weights.keys():
            rating = ant.get_task_cost(vertex)
            weight = temp_weights[vertex]
            sum_cost += ((weight **alpha) * (rating ** beta))
            
        for vertex in temp_weights.keys():
            rating = ant.get_task_cost(vertex)
            weight = temp_weights[vertex]
            probs[vertex] = prob_fx(rating,weight,sum_cost)
            
            
        #Chooses the next film based on probabilities 
        return random.choices(list(probs.keys()), weights= list(probs.values()), k=1)
            
                
        
                

In [31]:
class Task:
    title = ""
    genres = ""
    mID = None
    
    def __init__(self, movie,movie_details):
        self.mId = movie
        self.title = movie_details["title"]
        self.genres = movie_details["genres"]
    
    

In [32]:
class Ant_Graph:
    
    graph_edge_weights = {}
    verticies   = None
    trail_max_len = 0
    num_ants = 0 
    decay = 0 
    ants = None
    best_recommendations = []
    
    def __init__(self, movie_list, num_ants,num_movies_wanted,antID_list, filename = "./generation_saves/ratings_gen_50.csv", alpha =0.6, beta= 0.4, decay = 0.05):
        self.num_ants = num_ants
        self.trail_max_len = num_movies_wanted
        self.vertices = [Task(m) for m in movie_list]
        self.decay = decay
        
        for movieID in movie_list:
            if movieID not in graph_edge_weights.keys():
                self.graph_edge_weights[movieID] = {}
            for m in movie_list:
                if (m != movieID):
                    self.graph_edge_weights[movieID][m] = 0.5
        
        #creating a start node that does not belong to a movie, all agents will start there
        for movieID in movie_list:
            self.graph_edge_weights[0] = movieID
            
        self.ants = AntColony(antID_list,filename,alpha,beta)
            
                    
                    
                    
    def update_edge_weights (tours):
        
        # we will chqnge the pheremones based on how much the user enjoyed that set ("trail") of movies
        # it will be calculated as   cumulative_trail_Raitings/ total_possible ratings, bound between 0 and 1 
        prev = 0 
        cumulative_ratings =  {}
        for i,tour in enumerate(tours):
            cumulative_ratings[prev] = {}
            cumulative_ratings[prev][curr] = 0
            #represents the edge weight for traversal from one movie to another
            for movie in tour:
                curr = movie+1
                rating = ants.get_ant_rating(i,movie)
                cumulative_ratings [prev][curr] += rating
                
            prev = curr
            
        for startnode in cumulative_ratings.keys():
            for nextnode in cumulative_ratings[startnode].keys():
                graph_edge_weights[startnode][nextnode] *= (1-decay)
                graph_edge_weights[startnode][nextnode] += (cumulative_ratings[startnode][nextnode] / (5*trail_max_len))
    

    def compute_best_rec (exclude_list):
        temp_list = []
        while (len(temp_list)<10):
            test_ant = Ant([])
            nextid = determine_next_movie(test_ant,graph_edge_weights)
            test_ant.watch_movie(nextid)
            if (nextid not in exclude_list):
                temp_list.append(nextid)
        
        return temp_list
    
    def best_recomendations ():
        return best_recommendations
    
    def one_iter (): 
        print("running one iteration: \n -------------------------------")
        for step in range(0,trail_max_len):
            ants.move_all_ants()
            update_edge_weights(ants.return_all_tours())
            ants.clear()
        
        ## determine best recomendations 
        best_recommendations = compute_best_rec()
            
                
        
        

In [33]:
def compute_simillarity_ratings (l1,l2):
    l1_dict = dict(l1)
    l2_dict = dict(l2)
    simillarity = 0 
    
    genre_tracker1 = create_genre_tracker ()
    genre_tracker2 = create_genre_tracker ()
    
    l2_keys = list(l2_dict.keys())
    for key in l1_dict.keys():
        m1_genres = movies.loc[movies['movieId'] == key  ,'genres'].values[0].split("|")
        for mg in m1_genres:
            genre_tracker1 [mg] +=1
        if key in l2_keys:
            r1 = l1_dict [key]
            r2 = l2_dict [key]
            if (r1 < r2):
                simillarity += 100*(r1/r2)
            else:
                simillarity+= 100*(r2/r1) 
                
    for key in l2_keys:
        m2_genres = movies.loc[movies['movieId'] == key  ,'genres'].values[0].split("|")
        for mg in m2_genres:
            genre_tracker2 [mg] +=1
            
    genre_simillarity =  0 
    for g in genres:
        tempg1 = genre_tracker1 [g]
        tempg2 = genre_tracker2 [g]
        if (tempg1 > 0 or tempg2 >0):
            genre_simillarity += (1000* (min(tempg1,tempg2)/max(tempg1,tempg2))* min (tempg1,tempg2))
        
    return genre_simillarity + simillarity

In [34]:
def compute_simillarity_vector (ratings_df, userlist):
    simillarity_dict = {}
    
    for index, row in ratings_df.iterrows():
        user2list = ast.literal_eval (row["ratings_list"])
        simillarity_dict [row["userId"]] = compute_simillarity_ratings (userlist,user2list)
        
    return [[(k,v) for k ,v in simillarity_dict.items()]]

In [35]:
def select_similar_users(num_agents,user_ratings, userlist, randomize = False):
    if ( not randomize):
        sim_vec = compute_simillarity_vector (user_ratings, userlist) 
        sim_vec.sort(key=lambda pair: pair[1])
        return [a[0] for a in sim_vec[:num_agents]]
    else: 
        userids = list (range(1,len(user_ratings.index)))
        return random.sample(userids, num_agents);

In [36]:
def main (num_agents, num_iter, num_movies, userlist = [],rating_fn = "./generation_saves/ratings_gen_50.csv", simillarility_fn = "./simillarity_matrix_normalized.csv", alpha= 0.7,beta =0.3,decay=0.05):
    # import datasets
    sim_df  = pd.read_csv(simillarility_fn)
    movies  = pd.read_csv(f"{Dataset_folder}/movies.csv")
    user_ratings = pd.read_csv(rating_fn)
    
    ## if userlist is empty pick 50 random users as agents
    rand = (len(userlist) == 0)
        
    ## if not empty userlist will be the most simillar users 
    users = select_similar_users(num_agents, user_ratings, userlist, randomize = rand)
    
    ## determining all the movies that these users watched
    movie_list = []
    tasks = []
    for uID in users:
        film_list = ast.literal_eval(user_ratings.loc[user_ratings['userId'] == uID]["ratings_list"].tolist()[0])
        for film in film_list:
            if (film not in movie_list):
                movie_list.append(movie_list)
                tasks.append(Task(film,movies.loc[movies['movieId'] == movie]))
    
    ## define the graph
    graph = Ant_Graph(self, movie_list, num_agents ,num_movies, users, filename = rating_fn , alpha=alpha , beta= beta, decay = decay)
    for i in range (0,num_iter):
        graph.one_iter()
        
    movies_user_already_watched = [mid for mid,rate in userlist ]
    print(f"Recommended ID's: {graph.best_recomendations (movies_user_already_watched)}")

## Training and testing 

### training 

Training is rather trivial, all weneed to do is make an appropriate call to main()

### testing

Testing is a bit more difficult. We do not have a list of people who have volunteered to test the system by watching the recomendations and rating them so we have to get creative. We can set a few users in our existing dataset as "test users" then we will take the first k ratings of the user as input/"user history" and the n-k as "test ratings" . We will determine how good the recomendations are using the percentage of the reccomendations that appear in the "test ratings list and by comparing the ratings for movies that are in both. 

In [ ]:
# evaluation metric
def evaluate_reccomendations (recs, test_set)
    missing_recs =0 
    ratings = []
    for movieID in recs: 
        rating = next((y for x, y in test_set if x == movieID ), None)
        if (rating == None):
            missing_recs+=1
        else: 
            ratings.append(rating)
            
    percent_overlap = (len(recs) - missing_recs) / len(recs)
    rating_acccuracy = sum(ratings) / (5*len(ratings))
    
    return percent_overlap, rating_acccuracy